In [24]:
import mujoco
from mujoco import MjModel, MjData
from mujoco.viewer import launch_passive
from mujoco import viewer
import time

model = mujoco.MjModel.from_xml_path("tesbot.urdf")
data = mujoco.MjData(model)

In [25]:
with viewer.launch_passive(model, data) as viewer:
    for _ in range(100):
        mujoco.mj_step(model, data)
        viewer.sync()  # Ensure the viewer updates each frame
        time.sleep(model.opt.timestep)  # Real-time step duration
        